In [1]:
!nvidia-smi

Sat Nov 26 06:45:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    28W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!wget https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv

--2022-11-26 06:45:10--  https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 503663 (492K) [text/plain]
Saving to: ‘spam.csv.2’

spam.csv.2          100%[===================>] 491.86K  --.-KB/s    in 0.004s  

2022-11-26 06:45:10 (131 MB/s) - ‘spam.csv.2’ saved [503663/503663]



In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import regex as re
from tqdm import tqdm
from transformers import AutoTokenizer, TFAutoModel, AutoConfig

In [5]:
data = pd.read_csv("/content/spam.csv", encoding = "ISO-8859-1")

In [6]:
data.drop(["Unnamed: 2", "Unnamed: 3","Unnamed: 4"], axis=1, inplace=True)

In [7]:
data

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [8]:
data['gt'] = data['v1'].map({'ham':0,'spam':1})
data['text'] = data.v2.copy()
data['text'] = data.text.apply(lambda x: x.lower())

In [9]:
data.head()

,v1,v2,gt,text
0,ham,"Go until jurong point, crazy.. Available only ...",0,"go until jurong point, crazy.. available only ..."
1,ham,Ok lar... Joking wif u oni...,0,ok lar... joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,0,u dun say so early hor... u c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,"nah i don't think he goes to usf, he lives aro..."


In [10]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data["gt"], test_size=0.2, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [11]:
sentences=data['text']
labels=data['gt']
len(sentences),len(labels)

(5572, 5572)

# 1. Chạy mô hình bert-base-cased để xem kết quả thế nào

In [12]:
MAX_LENGTH = 128
NUM_CLASSES = len(data["gt"].unique())
MODEL_NAME = 'bert-base-cased'
MODEL_CONFIG = AutoConfig.from_pretrained(MODEL_NAME)

In [36]:
def Tokenizer(sentences,MODEL_NAME):
  tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, do_lower_case=False, add_special_tokens=True,
                                            max_length=MAX_LENGTH, pad_to_max_length=True, use_fast=True)
  input_ids=[]
  attention_masks=[]

  for sent in tqdm(sentences):
      bert_inp=tokenizer.encode_plus(sent, add_special_tokens = True ,max_length=MAX_LENGTH, pad_to_max_length = True, return_attention_mask = True)
      input_ids.append(bert_inp['input_ids'])
      attention_masks.append(bert_inp['attention_mask'])

  input_ids=np.asarray(input_ids)
  attention_masks=np.array(attention_masks)
  return input_ids, attention_masks

In [14]:
input_ids_train, attention_masks_train = Tokenizer(X_train,MODEL_NAME)
input_ids_test, attention_masks_test = Tokenizer(X_test,MODEL_NAME)
input_ids_valid, attention_masks_vaild = Tokenizer(X_valid,MODEL_NAME)

  0%|          | 0/3342 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 1115/1115 [00:00<00:00, 8410.80it/s]


In [35]:
def create_model(MODEL_NAME,config=MODEL_CONFIG):
    config.output_hidden_states = False
    transformer_model = TFAutoModel.from_pretrained(MODEL_NAME, config=config)

    input_ids_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='input_token', dtype='int32')
    input_masks_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='masked_token', dtype='int32') 
    embedding_layer = transformer_model(input_ids_in, input_masks_in)[0][:, 0, :]

    X = tf.keras.layers.Dense(20, activation='relu')(embedding_layer)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(10, activation='relu')(X)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(1, activation='sigmoid')(X)
    model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = X)
    return model

In [16]:
model = create_model(MODEL_NAME)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_token[0][0]',            
                                thPoolingAndCrossAt               'masked_token[0][0]']           
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                           

In [18]:
model.compile(loss="binary_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(learning_rate=5e-6),
              metrics=["accuracy"])

In [20]:
mc = tf.keras.callbacks.ModelCheckpoint(filepath="./best_model.hdf5",
                                        monitor="val_accuracy",
                                        verbos=1,
                                        save_best_only=True,
                                        mode="max")
history = model.fit([input_ids_train, attention_masks_train], 
                    y_train,
                    batch_size=64, 
                    epochs=5,
                    callbacks=[mc],
                    validation_data=([input_ids_valid, attention_masks_vaild], y_valid)
                    )

Epoch 1/5
53/53 [==============================] - 94s 2s/step - loss: 0.1897 - accuracy: 0.9294 - val_loss: 0.0669 - val_accuracy: 0.9794
Epoch 2/5
53/53 [==============================] - 91s 2s/step - loss: 0.0898 - accuracy: 0.9737 - val_loss: 0.0531 - val_accuracy: 0.9848
Epoch 3/5
53/53 [==============================] - 85s 2s/step - loss: 0.0783 - accuracy: 0.9725 - val_loss: 0.0553 - val_accuracy: 0.9839
Epoch 4/5
53/53 [==============================] - 85s 2s/step - loss: 0.0545 - accuracy: 0.9788 - val_loss: 0.0547 - val_accuracy: 0.9848
Epoch 5/5
53/53 [==============================] - 91s 2s/step - loss: 0.0472 - accuracy: 0.9835 - val_loss: 0.0570 - val_accuracy: 0.9857


In [21]:
model.evaluate([input_ids_test, attention_masks_test], y_test)

35/35 [==============================] - 9s 264ms/step - loss: 0.0441 - accuracy: 0.9874


[0.04406029358506203, 0.9874439239501953]

# 2. Thay Bert bằng xlnet-base-cased để chạy thử xem kết quả thế nào

In [37]:
MAX_LENGTH = 128
NUM_CLASSES = len(data["gt"].unique())
MODEL_NAME = 'xlnet-base-cased'
MODEL_CONFIG = AutoConfig.from_pretrained(MODEL_NAME)

In [38]:
input_ids_train, attention_masks_train = Tokenizer(X_train,MODEL_NAME)
input_ids_test, attention_masks_test = Tokenizer(X_test,MODEL_NAME)
input_ids_valid, attention_masks_vaild = Tokenizer(X_valid,MODEL_NAME)

100%|██████████| 1115/1115 [00:00<00:00, 6512.24it/s]


In [49]:
model = create_model(MODEL_NAME)

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


In [25]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model_1 (TFBertModel)  TFBaseModelOutputWi  108310272   ['input_token[0][0]',            
                                thPoolingAndCrossAt               'masked_token[0][0]']           
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                         

In [50]:
model.compile(loss="binary_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(learning_rate=5e-6),
              metrics=["accuracy"])

In [51]:
mc = tf.keras.callbacks.ModelCheckpoint(filepath="./best_model.hdf5",
                                        monitor="val_accuracy",
                                        verbos=1,
                                        save_best_only=True,
                                        mode="max")
history = model.fit([input_ids_train, attention_masks_train], 
                    y_train,
                    batch_size=32, 
                    epochs=5,
                    callbacks=[mc],
                    validation_data=([input_ids_valid, attention_masks_vaild], y_valid)
                    )

Epoch 1/5


105/105 [==============================] - 138s 1s/step - loss: 0.5226 - accuracy: 0.7499 - val_loss: 0.1870 - val_accuracy: 0.8906
Epoch 2/5
105/105 [==============================] - 119s 1s/step - loss: 0.2692 - accuracy: 0.8501 - val_loss: 0.1355 - val_accuracy: 0.8978
Epoch 3/5
105/105 [==============================] - 119s 1s/step - loss: 0.1972 - accuracy: 0.9069 - val_loss: 0.0777 - val_accuracy: 0.9839
Epoch 4/5
105/105 [==============================] - 119s 1s/step - loss: 0.1548 - accuracy: 0.9324 - val_loss: 0.0595 - val_accuracy: 0.9865
Epoch 5/5
105/105 [==============================] - 113s 1s/step - loss: 0.1215 - accuracy: 0.9527 - val_loss: 0.0644 - val_accuracy: 0.9848


In [52]:
model.evaluate([input_ids_test, attention_masks_test], y_test)

35/35 [==============================] - 11s 326ms/step - loss: 0.0399 - accuracy: 0.9892


[0.03985185921192169, 0.9892376661300659]

# 3.Theo như bài tập số 2 ở những layers cuối hãy thay thành các FC layers có số units lần lượt là 50, 30, 10

In [43]:
def create_model_3(MODEL_NAME,config=MODEL_CONFIG):
    config.output_hidden_states = False
    transformer_model = TFAutoModel.from_pretrained(MODEL_NAME, config=config)

    input_ids_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='input_token', dtype='int32')
    input_masks_in = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='masked_token', dtype='int32') 
    embedding_layer = transformer_model(input_ids_in, input_masks_in)[0][:, 0, :]

    X = tf.keras.layers.Dense(50, activation='relu')(embedding_layer)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(30, activation='relu')(X)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(10, activation='relu')(X)
    X = tf.keras.layers.Dropout(0.2)(X)

    X = tf.keras.layers.Dense(1, activation='sigmoid')(X)
    model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = X)
    return model

In [44]:
model = create_model_3(MODEL_NAME)
model.compile(loss="binary_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(learning_rate=5e-6),
              metrics=["accuracy"])

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.


In [45]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 128)]        0           []                               
                                                                                                  
 tfxl_net_model_1 (TFXLNetModel  TFXLNetModelOutput(  116718336  ['input_token[0][0]',            
 )                              last_hidden_state=(               'masked_token[0][0]']           
                                None, 128, 768),                                                  
                                 mems=((128, None,                                          

In [46]:
mc = tf.keras.callbacks.ModelCheckpoint(filepath="./best_model.hdf5",
                                        monitor="val_accuracy",
                                        verbos=1,
                                        save_best_only=True,
                                        mode="max")
history = model.fit([input_ids_train, attention_masks_train], 
                    y_train,
                    batch_size=32, 
                    epochs=5,
                    callbacks=[mc],
                    validation_data=([input_ids_valid, attention_masks_vaild], y_valid)
                    )

Epoch 1/5


105/105 [==============================] - 135s 1s/step - loss: 0.3878 - accuracy: 0.8223 - val_loss: 0.2230 - val_accuracy: 0.8753
Epoch 2/5
105/105 [==============================] - 119s 1s/step - loss: 0.2408 - accuracy: 0.8806 - val_loss: 0.1436 - val_accuracy: 0.9614
Epoch 3/5
105/105 [==============================] - 119s 1s/step - loss: 0.1540 - accuracy: 0.9345 - val_loss: 0.0801 - val_accuracy: 0.9803
Epoch 4/5
105/105 [==============================] - 119s 1s/step - loss: 0.1091 - accuracy: 0.9596 - val_loss: 0.0783 - val_accuracy: 0.9857
Epoch 5/5
105/105 [==============================] - 113s 1s/step - loss: 0.0915 - accuracy: 0.9707 - val_loss: 0.0656 - val_accuracy: 0.9848


In [48]:
model.evaluate([input_ids_test, attention_masks_test], y_test)

35/35 [==============================] - 12s 327ms/step - loss: 0.0574 - accuracy: 0.9857


[0.057411886751651764, 0.9856502413749695]